In [253]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from typing import Iterable, Callable
from questionanswering import squad

In [254]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [255]:
df = squad.parse_json_file("input/train-v2.0.json")
questions = list(df["question"])
contexts = list(df["context"])
df.info()

100%|████████████████████████████████| 442/442 [00:00<00:00, 683.69it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            130319 non-null  object
 1   title         130319 non-null  object
 2   question      130319 non-null  object
 3   answer_text   130319 non-null  object
 4   context       130319 non-null  object
 5   answer_start  130319 non-null  int16 
dtypes: int16(1), object(5)
memory usage: 5.2+ MB


In [256]:
assert len(df) == 130319
assert len(df) == len(set(df["id"]))
df.head()

,id,title,question,answer_text,context,answer_start
0,56be85543aeaaa14008c9063,Beyoncé,When did Beyonce start becoming popular?,in the late 1990s,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",269
1,56be85543aeaaa14008c9065,Beyoncé,What areas did Beyonce compete in when she was growing up?,singing and dancing,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",207
2,56be85543aeaaa14008c9066,Beyoncé,When did Beyonce leave Destiny's Child and become a solo singer?,2003,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",526
3,56bf6b0f3aeaaa14008c9601,Beyoncé,In what city and state did Beyonce grow up?,"Houston, Texas","Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",166
4,56bf6b0f3aeaaa14008c9602,Beyoncé,In which decade did Beyonce become famous?,late 1990s,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Bil

In [257]:
s = pd.Series(np.where(df["answer_start"]==-1, True, False))
assert s.value_counts()[True] == 43498
s.value_counts(normalize=True)

False    0.666219
True     0.333781
dtype: float64

In [258]:
df[df["answer_start"] == -1].sample(5).head(5)

,id,title,question,answer_text,context,answer_start
103649,5acd7b6807355d001abf4420,Railway_electrification_system,Many newer sections in tunnels are energised to what voltage?,,"The London Underground in England is one of the few networks that uses a four-rail system. The additional rail carries the electrical return that, on third rail and overhead networks, is provided by the running rails. On the London Underground, a top-contact third rail is beside the track, energized at +420v DC, and a top-contact fourth rail is located centrally between the running rails at −210v DC, which combine to provide a traction voltage of 630v DC. London Underground is now upgrading its fourth rail system to 750v DC with a positive conductor rail energised to +500v DC and a negative conductor rail energised to -250v DC. However, many older sections in tunnels are still energised to 630v DC. The same system was used for Milan's earliest underground line, Milan Metro's line 1, whose more recent lines use an overhead catenary or a third rail.",-1
122352,5ad141f4645df0001a2d140c,Separation_of_church_and_state_in_the_United_States,What does the second amendment to the US Constitution state?,,"The first amendment to the US Constitution states ""Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof"" The two parts, known as the ""establishment clause"" and the ""free exercise clause"" respectively, form the textual basis for the Supreme Court's interpretations of the ""separation of church and state"" doctrine. Three central concepts were derived from the 1st Amendment which became America's doctrine for church-state separation: no coercion in religious matters, no expectation to support a religion against one's will, and religious liberty encompasses all religions. In sum, citizens are free to embrace or reject a faith, any support for religion - financial or physical - must be voluntary, and all religions are equal in the eyes of the law with no special preference or favoritism.",-1
33848,5a486fb084b8a4001a7e786d,Imperial_College_London,How many nonoperational and development properties were in the portfolio?,,"The College's endowment is sub-divided into three distinct portfolios: (i) Unitised Scheme – a unit trust vehicle for College, Faculties and Departments to invest endowments and unfettered income to produce returns for the long term; (ii) Non-Core Property – a portfolio containing around 120 operational and developmental properties which College has determined are not core to the academic mission; and (iii) Strategic Asset Investments – containing College's shareholding in Imperial Innovations and other restricted equity holdings. During the year 2014/15, the market value of the endowment increased by £78 million (18%) to £512.4 million on 31 July 2015.",-1
83015,5acd69f607355d001abf411f,Strasbourg,In what year did the construction begin of the Immeuble Louise Weiss?,,"In 1949, the city was chosen to be the seat of the Council of Europe with its European Court of Human Rights and European Pharmacopoeia. Since 1952, the European Parliament has met in Strasbourg, which was formally designated its official 'seat' at the Edinburgh meeting of the European Council of EU heads of state and government in December 1992. (This position was reconfirmed and given treaty status in the 1997 Treaty of Amsterdam). However, only the (four-day) plenary sessions of the Parliament are held in Strasbourg each month, with all other business being conducted in Brussels and Luxembourg. Those sessions take place in the Immeuble Louise Weiss, inaugurated in 1999, which houses the largest parliamentary assembly room in Europe and of any democratic institution in the world. Before that, the EP sessions had to take place in the main Council of Europe building, the Palace of Europe, whose unusual inner architecture had become a familiar sight to European TV audiences. In 1992, Strasbourg became the seat of the Franco

# Answer corrections

In [259]:
%%time
df.loc[df["id"]=="56bf7e603aeaaa14008c9681", "answer_text"] = "split with Luckett and Roberson"
df.loc[df["id"]=="56be973d3aeaaa14008c9123", "answer_text"] = "sixth"
df.loc[df["id"]=="56d3ac8e2ccc5a1400d82e1b", "answer_text"] = "operatic"
df.loc[df["id"]=="56cbdea66d243a140015edae", "answer_text"] = "seven"
df.loc[df["id"]=="56cbdea66d243a140015edae", "answer_start"] = 332
df.loc[df["id"]=="56cf609aaab44d1400b89187", "answer_text"] = "seven"
df.loc[df["id"]=="56cf609aaab44d1400b89187", "answer_start"] = 332
df.loc[df["id"]=="56d20a6ae7d4791d0090261a", "answer_text"] = "seven"
df.loc[df["id"]=="56d20a6ae7d4791d0090261a", "answer_start"] = 332
df.loc[df["id"]=="56ccde7862d2951400fa64d9", "answer_start"] = 790
df.loc[df["id"]=="56cd682162d2951400fa658e", "answer_start"] = 463
df.loc[df["id"]=="56ce726faab44d1400b88793", "answer_text"] = "three"
df.loc[df["id"]=="56ce726faab44d1400b88793", "answer_start"] = 302
df.loc[df["id"]=="56cc57466d243a140015ef24", "question"] = "in which year did the sales of iPhone exceed iPod?"
df.loc[df["id"]=="56cc57466d243a140015ef24", "answer_text"] = "2010"
df.loc[df["id"]=="56cc57466d243a140015ef24", "answer_start"] = 444
df.loc[df["id"]=="56ce750daab44d1400b887b4", "answer_start"] = 396
df.loc[df["id"]=="56cd5d3a62d2951400fa653e", "answer_text"] = "manually"
df.loc[df["id"]=="56cd8ffa62d2951400fa6723", "question"] = "what is Akiko Kōmoto's nationality?"
df.loc[df["id"]=="56cd8ffa62d2951400fa6723", "answer_text"] = "Japanese"
df.loc[df["id"]=="56cf2e15aab44d1400b88dc9", "answer_text"] = "twenty-fourth"
df.loc[df["id"]=="56cf2e15aab44d1400b88dcb", "answer_text"] = "fourth"
df.loc[df["id"]=="56cf2e15aab44d1400b88dcb", "answer_start"] = 113


Wall time: 545 ms


In [260]:
%%time
col = "qc_length"
df[col] = df["question"].str.len() + df["context"].str.len() 
df[col] = df[col].astype(np.int32)

Wall time: 122 ms


In [261]:
%%time
col = "a_length"
df[col] = df["answer_text"].str.len()
df[col] = df[col].astype(np.int32)

Wall time: 78 ms


In [262]:
def word_length(cols: Iterable) -> Callable:
    def f(row) -> int:
        res = 0
        for col in cols:
            res += len(row[col].split())
        return res
    
    return f

In [263]:
%%time
col = "qc_word_length"
df[col] = df.progress_apply(word_length(["question", "context"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████| 130319/130319 [00:02<00:00, 48088.64it/s]

Wall time: 2.72 s


In [264]:
%%time
col = "a_word_length"
df[col] = df.progress_apply(word_length(["answer_text"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|███████████████████████| 130319/130319 [00:01<00:00, 102271.86it/s]

Wall time: 1.28 s


In [265]:
pretrained_dir = "../pretrained/google/electra-small-discriminator"
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [266]:
pretrained_dir = "../pretrained/albert-base-v2"
sp_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(sp_tokenizer)}\n{sp_tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/albert-base-v2', vocab_size=30000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)})
['input_ids', 'token_type_ids', 'attention_mask']


In [267]:
pretrained_dir = "../pretrained/distilroberta-base"
bpe_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(bpe_tokenizer)}\n{bpe_tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/distilroberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
['input_ids', 'attention_mask']


In [268]:
%%time
x = tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_wp_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=130319
Wall time: 3min 11s


In [269]:
%%time
x = sp_tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_sp_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=130319
Wall time: 4min 2s


In [270]:
%%time
x = bpe_tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_bpe_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'attention_mask'])
len=130319
Wall time: 2min 48s


In [271]:
cols = ["qc_length", "a_length", "qc_word_length", "a_word_length", 
        "qc_wp_length", "qc_sp_length", "qc_bpe_length"]
df[cols].describe(percentiles=percentiles)

,qc_length,a_length,qc_word_length,a_word_length,qc_wp_length,qc_sp_length,qc_bpe_length
count,130319.000000,130319.000000,130319.000000,130319.000000,130319.000000,130319.000000,130319.000000
mean,812.810450,13.423983,129.508061,2.107099,170.726632,174.188384,172.696813
std,308.710726,20.016896,49.604341,3.144031,65.407215,67.240044,65.938716
min,178.000000,0.000000,25.000000,0.000000,35.000000,35.000000,36.000000
1%,254.000000,0.000000,40.180000,0.000000,55.000000,56.000000,57.000000
5%,390.000000,0.000000,62.000000,0.000000,84.000000,86.000000,86.000000
10%,531.000000,0.000000,82.000000,0.000000,108.000000,109.000000,109.000000
20%,596.000000,0.000000,95.000000,0.000000,124.000000,126.000000,125.000000
30%,642.000000,0.000000,102.000000,0.000000,135.000000,137.000000,136.000000
40%,693.000000,4.000000,111.000000,1.000000,146.000000,149.000000,148.000000


In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              130319 non-null  object
 1   title           130319 non-null  object
 2   question        130319 non-null  object
 3   answer_text     130319 non-null  object
 4   context         130319 non-null  object
 5   answer_start    130319 non-null  int16 
 6   qc_length       130319 non-null  int32 
 7   a_length        130319 non-null  int32 
 8   qc_word_length  130319 non-null  int32 
 9   a_word_length   130319 non-null  int32 
 10  qc_wp_length    130319 non-null  int16 
 11  qc_sp_length    130319 non-null  int16 
 12  qc_bpe_length   130319 non-null  int16 
dtypes: int16(4), int32(4), object(5)
memory usage: 8.0+ MB


In [273]:
%%time
df.to_parquet("output/train.parquet", index=False)

Wall time: 568 ms
